## Embeddings Similarity Evaluation

### imports

In [ ]:
from __future__ import print_function

import numpy as np
import operator
import easydict
import linecache
import ipywidgets as widgets
import ipywidgets

from  scipy.spatial.distance import cdist
from ipywidgets import interact, fixed


In [ ]:
embedding_type = "definition" # definition, graph, etc

args = easydict.EasyDict({
    "embedding_base_dir": f'gene_ontology/{embedding_type}_embedding',  # base directory to the Gene Ontology embeddings
    "go_subontology": 'BP',     # GO (Gene Ontology) subontology (BP, CC, MF)
    "embedding_dim": 100,       # embedding dimemnsion of interest
    "go_dir": 'gene_ontology/raw/',     # directory to the Gene Ontology 'go.obo' file
    "download_gene_ontology": False,    # download the latest version of gene ontology into the specified directory above
})

### Loading Gene Ontology<a id='3'></a>
[back to top](#top)<br>

In [ ]:
if args.download_gene_ontology:
    os.makedirs(args.go_dir, exist_ok=True)  # create 'data_loc' folder (if it does not exist already)
    print("Downloading the latest version of Gene Ontology into '{}'...".format(args.go_dir))
    url = 'http://current.geneontology.org/ontology/go.obo'
    r = requests.get(url, allow_redirects=True)
    open('{}/go.obo'.format(args.go_dir), 'wb').write(r.content)

print("Gene Ontology {}".format(linecache.getline('{}/go.obo'.format(args.go_dir), 2))) # Now: releases/2020-10-09

In [ ]:
"""Reading Gene Ontology to extract Terms and their Descriptive Names"""
with open("{}/go.obo".format(args.go_dir)) as f:
    content = f.readlines()
content = "".join([x for x in content])
content = content.split("[Typedef]")[0].split("[Term]")
print("Information of the last GO term in the file:\n~~~~~~~~~~~~~~~~~~~~~~~~~{}".format(content[-1]))

In [ ]:
go_term_dict = {}
for c in content:
    go_id = ''
    for l in c.split("\n"):
        # id
        if "id: GO:" in l[0:len("id: GO:")]:
            go_id = l.split("id: ")[1]
            go_term_dict[go_id] = {}
        # alt_id
        if "alt_id:" in l[0:len("alt_id")+1]:
            go_term_dict[go_id].setdefault("alt_id", []).append(l.split("alt_id: ")[1])
        # name
        if "name:" in l[0:len("name")+1]:
            go_term_dict[go_id]["name"] = l.split("name: ")[1]
        # namespace
        if "namespace:" in l[0:len("namespace")+1]:
            go_term_dict[go_id]["namespace"] = l.split("namespace: ")[1]
            
go_term_dict

In [ ]:
name_space_map = {'molecular_function': "MF", "biological_process": "BP", "cellular_component": "CC"}
go_to_name = {"MF":{}, "BP":{}, "CC":{}}
for go in go_term_dict:
    go_to_name[name_space_map[go_term_dict[go]['namespace']]][go] = go_term_dict[go]['name']

In [ ]:
go_to_name["MF"]

### Reading GO-term Embedding Vectors

In [ ]:
file_reader = open(f"{args.embedding_base_dir}/{args.embedding_dim}_dimensional/GO_{args.go_subontology}_Embeddings_{args.embedding_dim}D.emb", "r")
go_term_embeddings = {}
for line in file_reader:
    values = line.split()
    go_term = values[0]
    go_term_embeddings[go_term] = np.asarray(values[1:], dtype='float32')
file_reader.close()
print(f'Loaded {len(go_term_embeddings)} GO-term embedding vectors.')

### Computing GO-term Similarity (cosine)

In [ ]:
default = {"BP": "GO:0045210", "CC": "GO:0000127", "MF":"GO:0001883"}

@interact(GO_term = widgets.Combobox(options=list(go_to_name[args.go_subontology]), value=default[args.go_subontology], description='GO term:'), 
          metric = ipywidgets.fixed("cosine"),
          embedding_matrix = ipywidgets.fixed(go_term_embeddings),
          n_top = widgets.IntSlider(20, min=5, max=50, description="N top terms:"))
def most_similar_concepts(GO_term, metric, embedding_matrix, n_top):
    if GO_term not in embedding_matrix:
        print("Error: The specified input GO term does not exists in the embedding version of GO.")
        return
    XA = embedding_matrix[GO_term]
    Cosine_dic = {}
    for concept, XB in embedding_matrix.items():
        Y = cdist([XA], [XB], "cosine")
        Cosine_dic[concept] = Y[0][0]
    i = 0
    print(f"\nThe top {n_top} most similar {args.go_subontology} terms to the query term '{GO_term}' are (used {metric} metric):")
    print("~"*90)
    print(" #      GO term      Score    Label")
    print("-"*90)
    for i, (concept, cosine) in enumerate(sorted(Cosine_dic.items(), key=operator.itemgetter(1))[:n_top+1]): # +1 to consider the term itself
        if i==0: print(f' Q  >>  {concept}   {np.abs(cosine):.04f}   {go_to_name[args.go_subontology][concept]}')
        else: print(f'{i:2.0f}  >>  {concept}   {np.abs(cosine):.04f}   {go_to_name[args.go_subontology][concept]}')


-----